# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 15 2022 1:09PM,246541,12,HH18,Hush Hush,Executing
1,Sep 15 2022 12:58PM,246540,16,EMERSA-8530559,"Emersa Waterbox, LLC",Released
2,Sep 15 2022 12:51PM,246537,20,ALU0007925,Alumier Labs Inc.,Released
3,Sep 15 2022 12:51PM,246537,20,ALU0007928,Alumier Labs Inc.,Executing
4,Sep 15 2022 12:51PM,246537,20,ALUR031061309,Alumier Labs Inc.,Executing
5,Sep 15 2022 12:51PM,246537,20,ALUR229532274,Alumier Labs Inc.,Executing
6,Sep 15 2022 12:51PM,246537,20,ALUR441573843,Alumier Labs Inc.,Executing
7,Sep 15 2022 12:51PM,246537,20,ALU0007933,Alumier Labs Inc.,Released
8,Sep 15 2022 12:39PM,246536,10,WT5339 Out,Eywa Pharma Inc.,Executing
9,Sep 15 2022 12:17PM,246532,10,0085983034,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,246536,Executing,1
35,246537,Executing,4
36,246537,Released,2
37,246540,Released,1
38,246541,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246532,NaN,NaN,16.0
246536,NaN,1.0,NaN
246537,NaN,4.0,2.0
246540,NaN,NaN,1.0
246541,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246067,0.0,1.0,0.0
246068,0.0,0.0,1.0
246319,0.0,0.0,1.0
246324,0.0,0.0,1.0
246325,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246067,0,1,0
246068,0,0,1
246319,0,0,1
246324,0,0,1
246325,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246067,0,1,0
1,246068,0,0,1
2,246319,0,0,1
3,246324,0,0,1
4,246325,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246067,,1,
1,246068,,,1
2,246319,,,1
3,246324,,,1
4,246325,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 15 2022 1:09PM,246541,12,Hush Hush
1,Sep 15 2022 12:58PM,246540,16,"Emersa Waterbox, LLC"
2,Sep 15 2022 12:51PM,246537,20,Alumier Labs Inc.
8,Sep 15 2022 12:39PM,246536,10,Eywa Pharma Inc.
9,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation
25,Sep 15 2022 12:16PM,246530,20,"ACI Healthcare USA, Inc."
26,Sep 15 2022 12:06PM,246529,10,Yusen – 3D Matrix
27,Sep 15 2022 12:06PM,246528,10,"Nextsource Biotechnology, LLC"
29,Sep 15 2022 11:56AM,246526,10,Emerginnova
32,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 15 2022 1:09PM,246541,12,Hush Hush,,1,
1,Sep 15 2022 12:58PM,246540,16,"Emersa Waterbox, LLC",,,1
2,Sep 15 2022 12:51PM,246537,20,Alumier Labs Inc.,,4,2
3,Sep 15 2022 12:39PM,246536,10,Eywa Pharma Inc.,,1,
4,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation,,,16
5,Sep 15 2022 12:16PM,246530,20,"ACI Healthcare USA, Inc.",,1,
6,Sep 15 2022 12:06PM,246529,10,Yusen – 3D Matrix,,,1
7,Sep 15 2022 12:06PM,246528,10,"Nextsource Biotechnology, LLC",,,2
8,Sep 15 2022 11:56AM,246526,10,Emerginnova,,,3
9,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 1:09PM,246541,12,Hush Hush,,1,
1,Sep 15 2022 12:58PM,246540,16,"Emersa Waterbox, LLC",1,,
2,Sep 15 2022 12:51PM,246537,20,Alumier Labs Inc.,2,4,
3,Sep 15 2022 12:39PM,246536,10,Eywa Pharma Inc.,,1,
4,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation,16,,
5,Sep 15 2022 12:16PM,246530,20,"ACI Healthcare USA, Inc.",,1,
6,Sep 15 2022 12:06PM,246529,10,Yusen – 3D Matrix,1,,
7,Sep 15 2022 12:06PM,246528,10,"Nextsource Biotechnology, LLC",2,,
8,Sep 15 2022 11:56AM,246526,10,Emerginnova,3,,
9,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 1:09PM,246541,12,Hush Hush,,1,
1,Sep 15 2022 12:58PM,246540,16,"Emersa Waterbox, LLC",1,,
2,Sep 15 2022 12:51PM,246537,20,Alumier Labs Inc.,2,4,
3,Sep 15 2022 12:39PM,246536,10,Eywa Pharma Inc.,,1,
4,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation,16,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 1:09PM,246541,12,Hush Hush,NaN,1.0,NaN
1,Sep 15 2022 12:58PM,246540,16,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,Sep 15 2022 12:51PM,246537,20,Alumier Labs Inc.,2.0,4.0,NaN
3,Sep 15 2022 12:39PM,246536,10,Eywa Pharma Inc.,NaN,1.0,NaN
4,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation,16.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 1:09PM,246541,12,Hush Hush,0.0,1.0,0.0
1,Sep 15 2022 12:58PM,246540,16,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,Sep 15 2022 12:51PM,246537,20,Alumier Labs Inc.,2.0,4.0,0.0
3,Sep 15 2022 12:39PM,246536,10,Eywa Pharma Inc.,0.0,1.0,0.0
4,Sep 15 2022 12:17PM,246532,10,ISDIN Corporation,16.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3697667,62.0,9.0,0.0
12,246541,0.0,1.0,0.0
15,493029,6.0,0.0,0.0
16,1478974,6.0,0.0,0.0
17,246519,1.0,0.0,0.0
19,492982,2.0,0.0,0.0
20,739564,2.0,7.0,3.0
21,1477436,5.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3697667,62.0,9.0,0.0
1,12,246541,0.0,1.0,0.0
2,15,493029,6.0,0.0,0.0
3,16,1478974,6.0,0.0,0.0
4,17,246519,1.0,0.0,0.0
5,19,492982,2.0,0.0,0.0
6,20,739564,2.0,7.0,3.0
7,21,1477436,5.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,62.0,9.0,0.0
1,12,0.0,1.0,0.0
2,15,6.0,0.0,0.0
3,16,6.0,0.0,0.0
4,17,1.0,0.0,0.0
5,19,2.0,0.0,0.0
6,20,2.0,7.0,3.0
7,21,5.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,62.0
1,12,Released,0.0
2,15,Released,6.0
3,16,Released,6.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
Executing,9.0,1.0,0.0,0.0,0.0,0.0,7.0,1.0
Released,62.0,0.0,6.0,6.0,1.0,2.0,2.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,9.0,1.0,0.0,0.0,0.0,0.0,7.0,1.0
2,Released,62.0,0.0,6.0,6.0,1.0,2.0,2.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,9.0,1.0,0.0,0.0,0.0,0.0,7.0,1.0
2,Released,62.0,0.0,6.0,6.0,1.0,2.0,2.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()